In [1]:
import os
import torch
import numpy as np
import pandas as pd

import scanpy as sc
from anndata import AnnData

only_not_interact_gene=True

result_dir = "../edges/"

In [2]:
data_dir = "../../data/NSCLC/processed1/"

ligands_info = torch.load("/".join(data_dir.split("/")[:-2]) + "/ligands.pth")
genes = torch.load("/".join(data_dir.split("/")[:-2]) + "/genes.pth")

samples=['Lung13', 'Lung6', 'Lung5_Rep1', 'Lung5_Rep3', 'Lung5_Rep2', 'Lung9_Rep1', 'Lung9_Rep2', 'Lung12']

#cell_types=['lymphocyte', 'fibroblast', 'Mcell', 'epithelial', 'tumors', 'endothelial', 'mast', 'neutrophil']
cell_types=['B-cell', 'NK', 'T CD4 memory', 'T CD4 naive', 'T CD8 memory', 'T CD8 naive', 'Treg', 'endothelial', 'epithelial', 'fibroblast', 'mDC', 'macrophage', 'mast', 'monocyte', 'neutrophil', 'pDC', 'plasmablast', 'tumor 12', 'tumor 13', 'tumor 5', 'tumor 6', 'tumor 9']

# Sex metadata
sex = {
    'Lung5_Rep1': 'F', 'Lung5_Rep2': 'F', 'Lung5_Rep3': 'F',
    'Lung6': 'M',
    'Lung9_Rep1': 'F', 'Lung9_Rep2': 'F',
    'Lung12': 'F',
    'Lung13': 'M'
}

# Histological diagnosis metadata
histological_diagnosis = {
    'Lung5_Rep1': 'adenocarcinoma', 'Lung5_Rep2': 'adenocarcinoma', 'Lung5_Rep3': 'adenocarcinoma',
    'Lung6': 'squamous cell carcinoma',
    'Lung9_Rep1': 'adenocarcinoma', 'Lung9_Rep2': 'adenocarcinoma',
    'Lung12': 'adenocarcinoma',
    'Lung13': 'adenocarcinoma'
}

# Grade metadata
grade = {
    'Lung5_Rep1': 'G1', 'Lung5_Rep2': 'G1', 'Lung5_Rep3': 'G1',
    'Lung6': 'G2',
    'Lung9_Rep1': 'G3', 'Lung9_Rep2': 'G3',
    'Lung12': 'G3',
    'Lung13': 'G1'
}

# T component of TNM
t_classification = {
    'Lung5_Rep1': 'T2a', 'Lung5_Rep2': 'T2a', 'Lung5_Rep3': 'T2a',
    'Lung6': 'T2b',
    'Lung9_Rep1': 'T3', 'Lung9_Rep2': 'T3',
    'Lung12': 'T4',
    'Lung13': 'T3'
}

# N component of TNM
n_classification = {
    'Lung5_Rep1': 'N2', 'Lung5_Rep2': 'N2', 'Lung5_Rep3': 'N2',
    'Lung6': 'N2',
    'Lung9_Rep1': 'N1', 'Lung9_Rep2': 'N1',
    'Lung12': 'N0',
    'Lung13': 'N0'
}

# M component of TNM
m_classification = {
    'Lung5_Rep1': 'M0', 'Lung5_Rep2': 'M0', 'Lung5_Rep3': 'M0',
    'Lung6': 'M0',
    'Lung9_Rep1': 'M0', 'Lung9_Rep2': 'M0',
    'Lung12': 'M0',
    'Lung13': 'M0'
}
# Stage metadata
stage = {
    'Lung5_Rep1': 'IIIA', 'Lung5_Rep2': 'IIIA', 'Lung5_Rep3': 'IIIA',
    'Lung6': 'IIIA',
    'Lung9_Rep1': 'IIIA', 'Lung9_Rep2': 'IIIA',
    'Lung12': 'IIIA',
    'Lung13': 'IIB'
}

meta={"sex":sex,"histological_diagnosis":histological_diagnosis,"grade":grade,"t_classification":t_classification,"n_classification":n_classification,"stage":stage}

In [3]:
cell_type_pair_sequence=[]
for cell_typei in cell_types:
    for cell_typej in cell_types:
        cell_type_pair_sequence.append(cell_typei+"__"+cell_typej)

def reshape_z_value(result_dict):
    results=[]
    for genei in genes+["all"]:
        resulti=np.zeros((len(cell_type_pair_sequence)))
        tmp=result_dict[genei]
        for j in range(len(tmp[0])):
            resulti[cell_type_pair_sequence.index(tmp[0][j])]=tmp[1][j]
        results.append(resulti)
    return np.stack(results,axis=0).transpose((-1,-2)) #(number_of_cell_type_pair,genes)

In [4]:
z_dir="./counts/"
if not os.path.exists(z_dir):
    os.system("mkdir "+z_dir)

def get_counts(sample):
    results = torch.load(result_dir + "edges_" + sample + ".pth")
    cell_type_name = results["cell_type_name"]
    cell_type_target = [cell_type_name[i][0] for i in range(len(cell_type_name))]
    types,counts=np.unique(cell_type_target,return_counts=True)
    counts=counts.tolist()
    types=types.tolist()
    counts1=[]
    for i in range(len(cell_types)):
        if cell_types[i] not in types:
            print(cell_types[i],"not in",sample,"with cell types:",types)
            counts1.append(0)
            continue
        counts1.append(counts[types.index(cell_types[i])])
    df=pd.DataFrame({"cell_type":cell_types,"counts":counts1})
    df.to_csv(z_dir+sample+".csv",index=False)

for i in range(len(samples)):
    samplei=samples[i]
    get_counts(samplei)
    print("finish counting:",samplei)

finish counting: Lung13
finish counting: Lung6
finish counting: Lung5_Rep1
finish counting: Lung5_Rep3
finish counting: Lung5_Rep2
finish counting: Lung9_Rep1
finish counting: Lung9_Rep2
finish counting: Lung12


In [5]:
# Statistics: regression

In [6]:
from scipy import stats
import statsmodels.api as sm
from scipy.stats import norm

def wald_test_z_score(X, y):
    model = sm.OLS(y, X).fit()
    z_scores = model.params / model.bse
    z_scores=np.where(model.bse==0,np.zeros_like(z_scores),z_scores)
    return z_scores

def judge_correlation(x, y):
    corr_xy = np.corrcoef(x, y)[0, 1]
    if corr_xy > 0:
        return 1
    elif corr_xy < 0:
        return -1
    else:
        return 0

def read_regression_multiple_wald(sample):
    global result_dir, cell_types, cell_type_pair_sequence, genes
    
    # Load the results
    results = torch.load(result_dir + "edges_" + sample + ".pth")
    
    # Extract relevant data
    attention_scores = results["attention_score"]  # Shape (B, 49, C)
    
    cell_type_names = np.array(results["cell_type_name"])  # Shape (B, 50)
    true_expression = results["y"]  # Shape (B, C)
    
    # Initialize a tensor to hold aggregated interaction strengths
    B, _, C = attention_scores.shape
    t = len(cell_types)
    aggregated_interactions = torch.zeros((B, t, C))
    
    # Map cell type names to indices
    cell_type_to_index = {ct: idx for idx, ct in enumerate(cell_types)}
    
    # Aggregate interaction strengths by cell type
    for b in range(B):
        for n in range(1, 50):  # Skip the first element, which is the target cell type
            neighbor_type = cell_type_names[b][n]
            if neighbor_type in cell_type_to_index:
                idx = cell_type_to_index[neighbor_type]
                aggregated_interactions[b, idx] += attention_scores[b, n-1]
    
    # Convert to numpy for statistical analysis
    aggregated_interactions_np = aggregated_interactions.numpy()
    true_expression_np = true_expression.numpy()
    
    # Prepare results array
    results_matrix = np.zeros((len(cell_types), len(cell_types), C))
    
    # Perform analysis
    for gene_index in range(C):
        for to_type in cell_types:
            mask = (cell_type_names[:, 0] == to_type)
            X = aggregated_interactions_np[mask, :, gene_index]
            y = true_expression_np[mask, gene_index]

            if y.shape[0]<=len(cell_types)+3 or len(np.unique(y))<=1:
                continue
            
            z_scores=wald_test_z_score(X, y)
            z_scores[np.sum(np.abs(X),axis=0)==0]=0
            
            X_proportion=np.abs(X)/np.sum(np.abs(X),axis=-1,keepdims=True)
            signs=[]
            for from_type in cell_types: 
                sign=judge_correlation(X_proportion[:,cell_types.index(from_type)],y)
                signs.append(sign)
            signs=np.array(signs)
            
            results_matrix[:,cell_types.index(to_type),gene_index] = z_scores*signs
            
    results_matrix[results_matrix>10]=10
    results_matrix[results_matrix<-10]=-10
    results_matrix=results_matrix.reshape(-1,C)
    return results_matrix

z_dir="./z_regressionp_multiple_wald/"
if not os.path.exists(z_dir):
    os.system("mkdir "+z_dir)
    
results=[]
cnt=0
for samplei in samples:
    print(cnt+1,len(samples))
    cnt=cnt+1
    tmp=read_regression_multiple_wald(samplei)
    tmp[np.isnan(tmp) | np.isinf(tmp)] = 0
    print(tmp.shape,np.max(tmp),np.min(tmp),np.mean(tmp),np.median(tmp))
    df=pd.DataFrame(data=tmp,columns=genes,index=cell_type_pair_sequence)
    df.to_csv(z_dir+samplei+".csv")
    results.append(tmp)
    print("regression:",samplei)

np.save(z_dir+"z_values.npy",np.stack(results,axis=0))

1 8
(484, 960) 10.0 -10.0 -0.30755670719885375 -0.24045469611883163
regression: Lung13
2 8


/tmp/ipykernel_1652216/3188875461.py:7: RuntimeWarning: invalid value encountered in divide
  z_scores = model.params / model.bse
/gpfs/gibbs/project/wang_zuoheng/xx244/conda_envs/torch/lib/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/gpfs/gibbs/project/wang_zuoheng/xx244/conda_envs/torch/lib/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


(484, 960) 10.0 -10.0 -0.4286311157078702 -0.43946635723114014
regression: Lung6
3 8
(484, 960) 10.0 -10.0 -0.12469368332488079 0.0
regression: Lung5_Rep1
4 8
(484, 960) 10.0 -10.0 -0.2167438345599821 -0.050584595650434494
regression: Lung5_Rep3
5 8
(484, 960) 10.0 -10.0 -0.03506762835438967 0.0
regression: Lung5_Rep2
6 8
(484, 960) 10.0 -10.0 -0.48095674048835485 -0.5175212025642395
regression: Lung9_Rep1
7 8
(484, 960) 10.0 -10.0 -0.6830186811160608 -0.704478532075882
regression: Lung9_Rep2
8 8


/tmp/ipykernel_1652216/3188875461.py:7: RuntimeWarning: divide by zero encountered in divide
  z_scores = model.params / model.bse


(484, 960) 10.0 -10.0 -0.11609876189975238 0.0
regression: Lung12


In [7]:
import pandas as pd

def read_regression_adapt(sample):
    # Assuming result_dir is a globally available directory path
    global result_dir, cell_types, cell_type_pair_sequence
    
    # Load the results
    results = torch.load(result_dir + "edges_" + sample + ".pth")
    
    # Extract relevant data
    attention_scores = results["attention_score"]  # Shape (B, 49, C)
    
    #proportion=torch.cumsum(results["attention_score"],dim=1)/8
    #y_pred=results["y_pred"].unsqueeze(dim=1)
    #attention_scores[torch.abs(proportion)>torch.abs(y_pred)*0.8]=0

    proportion=torch.abs(results["attention_score"])
    proportion=proportion/torch.sum(proportion,dim=1,keepdim=True)
    attention_scores[proportion<0.1]=0
    
    cell_type_names = np.array(results["cell_type_name"])  # Shape (B, 50)
    true_expression = results["y"]  # Shape (B, C)
    #print(calculate_mean_expression_by_cell_type(true_expression, cell_type_names[:,0], cell_types))
    
    # Initialize a tensor to hold aggregated interaction strengths
    B, _, C = attention_scores.shape
    t = len(cell_types)
    aggregated_interactions = torch.zeros((B, t, C))
    
    # Map cell type names to indices
    cell_type_to_index = {ct: idx for idx, ct in enumerate(cell_types)}
    
    # Aggregate interaction strengths by cell type
    for b in range(B):
        for n in range(1, 50):  # Skip the first element, which is the target cell type
            neighbor_type = cell_type_names[b][n]
            if neighbor_type in cell_type_to_index:
                idx = cell_type_to_index[neighbor_type]
                aggregated_interactions[b, idx] += attention_scores[b, n-1]
    
    aggregated_interactions1=torch.abs(aggregated_interactions)/torch.sum(torch.abs(aggregated_interactions),dim=1,keepdim=True)
    aggregated_interactions=torch.where(torch.sum(torch.abs(aggregated_interactions),dim=1,keepdim=True)==0,torch.zeros_like(aggregated_interactions),aggregated_interactions1)
    # Prepare to compute correlations for each cell type pair
    results_matrix = []
    
    for pair in cell_type_pair_sequence:
        from_type, to_type = pair.split("__")
        if from_type in cell_type_to_index:
            mask = (cell_type_names[:, 0] == to_type)
            filtered_interactions = aggregated_interactions[mask, cell_type_to_index[from_type]]
            filtered_expressions = true_expression[mask]
            if np.sum(mask)==0:
                results_matrix.append([0 for k in range(C)])
                continue
            
            # Calculate Pearson correlation coefficient for each gene
            corr_coeffs = []
            for i in range(C):
                gene_interactions = filtered_interactions[:, i]
                gene_expressions = filtered_expressions[:, i]
                if len(gene_interactions)<=10 or ((gene_interactions == gene_interactions[0]).all() or (gene_expressions == gene_expressions[0]).all()):
                    corr_coeffs.append(0)
                    continue
                r = torch.corrcoef(torch.stack((gene_interactions, gene_expressions)))[0, 1]
                n = gene_interactions.numel()
                z_value = r * ((n-2)**0.5) / (1 - r**2)**0.5
                if torch.isnan(z_value) or torch.isinf(z_value) or r==1:
                    print(from_type, to_type, np.sum((cell_type_names[:, 0] == to_type)))
                    print(r,z_value,gene_interactions,gene_expressions)
                    z_value=0
                corr_coeffs.append(float(z_value))
            results_matrix.append(corr_coeffs)
    
    # Convert results to a tensor of shape (t^2, C)
    results_tensor = np.array(results_matrix)
    results_tensor[results_tensor>10]=10
    results_tensor[results_tensor<-10]=-10
    results_matrix=np.nan_to_num(results_matrix)
    return results_tensor

z_dir="./z_regressionp_adapt/"
if not os.path.exists(z_dir):
    os.system("mkdir "+z_dir)
    
results=[]
cnt=0
for samplei in samples:
    print(cnt+1,len(samples))
    cnt=cnt+1
    tmp=read_regression_adapt(samplei)
    print(tmp.shape,np.max(tmp),np.min(tmp),np.mean(tmp),np.median(tmp))
    df=pd.DataFrame(data=tmp,columns=genes,index=cell_type_pair_sequence)
    df.to_csv(z_dir+samplei+".csv")
    results.append(tmp)
    print("regression:",samplei)

np.save(z_dir+"z_values.npy",np.stack(results,axis=0))

1 8
B-cell T CD4 naive 62
tensor(1.) tensor(inf) tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2573,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]) tensor([-0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170,
        -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170,
        -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170,
        -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170,
        -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170, -0.0170

In [8]:
import pandas as pd

def calcualte_z_neighbor(x,y,avg_cnti):
    p=torch.mean(y/avg_cnti)
    var=y.shape[0]*p*(1-p)/30
    if var==0:
        return 0
    return float(torch.mean(x-p))

def calculate_strength_spatial_neighbor_adapt(sample):
    # Assuming result_dir is a globally available directory path
    global result_dir, cell_types, cell_type_pair_sequence
    
    # Load the results
    results = torch.load(result_dir + "edges_" + sample + ".pth")
    cell_type_counts=pd.read_csv("./counts/"+sample+".csv")
    counts_all=float(np.sum(cell_type_counts.loc[:,"counts"].values))
    
    # Extract relevant data
    attention_scores = results["attention_score"]  # Shape (B, 49, C)
    
    #proportion=torch.cumsum(results["attention_score"],dim=1)/8
    #y_pred=results["y_pred"].unsqueeze(dim=1)
    #attention_scores[torch.abs(proportion)>torch.abs(y_pred)*0.8]=0

    proportion=torch.abs(results["attention_score"])
    proportion=proportion/torch.sum(proportion,dim=1,keepdim=True)
    attention_scores[proportion<0.05]=0
    
    expect_cnt_attention_scores=torch.where(attention_scores!=0,torch.ones_like(attention_scores),torch.zeros_like(attention_scores))
    
    cell_type_names = np.array(results["cell_type_name"])  # Shape (B, 50)
    true_expression = results["y"]  # Shape (B, C)
    pred_expression=results["y_pred"]
    
    cell_type_target=[cell_type_names[i][0] for i in range(len(cell_type_names))]
    type_exp_dict=np.load(data_dir + sample + "_TypeExp.npz", allow_pickle=True)
    type_exps=torch.Tensor(np.stack([type_exp_dict[cell_typei] for cell_typei in cell_type_target],axis=0))
    
    #true_expression=true_expression+type_exps
    #pred_expression=pred_expression+type_exps
    
    # Initialize a tensor to hold aggregated interaction strengths
    B, _, C = attention_scores.shape
    t = len(cell_types)
    aggregated_interactions = torch.zeros((B, t, C))
    expected_interactions = torch.zeros((B, t, C))
    
    # Map cell type names to indices
    cell_type_to_index = {ct: idx for idx, ct in enumerate(cell_types)}
    
    # Aggregate interaction strengths by cell type
    for b in range(B):
        for n in range(1, 50):  # Skip the first element, which is the target cell type
            neighbor_type = cell_type_names[b][n]
            if neighbor_type in cell_type_to_index:
                idx = cell_type_to_index[neighbor_type]
                aggregated_interactions[b, idx] += attention_scores[b, n-1]
                expected_interactions[b, idx]+=expect_cnt_attention_scores[b, n-1] 

    aggregated_interactions1=torch.abs(aggregated_interactions)/torch.sum(torch.abs(aggregated_interactions),dim=1,keepdim=True)
    aggregated_interactions=torch.where(torch.sum(torch.abs(aggregated_interactions),dim=1,keepdim=True)==0,torch.zeros_like(aggregated_interactions),aggregated_interactions1)

    for cell_typei in cell_types:
        mask = (cell_type_names[:, 0] == cell_typei)
        for genei in range(C):
            aggregated_interactions[mask,:,genei]=aggregated_interactions[mask,:,genei]/torch.sum(torch.abs(aggregated_interactions[mask,:,genei]))*aggregated_interactions[mask,:,genei].shape[0]
    
    # Prepare to compute correlations for each cell type pair
    results_matrix = []
    for pair in cell_type_pair_sequence:
        from_type, to_type = pair.split("__")
        if from_type in cell_type_to_index:
            mask = (cell_type_names[:, 0] == to_type)
            filtered_interactions = aggregated_interactions[mask, cell_type_to_index[from_type]]
            filtered_expected_interactions = expected_interactions[mask, cell_type_to_index[from_type]]
            filtered_expressions = true_expression[mask]
            filtered_pred=pred_expression[mask]

            avg_cnt=torch.mean(torch.sum(expected_interactions[mask],dim=1),dim=0)
            
            if np.sum(mask)==0:
                results_matrix.append([0 for k in range(C)])
                continue
            
            # Calculate Pearson correlation coefficient for each gene
            corr_coeffs = []
            for i in range(C):
                gene_interactions = filtered_interactions[:, i]
                gene_expressions = filtered_expressions[:, i]
                expectedi=filtered_expected_interactions[:, i]
                predi=filtered_pred[:,i]
                #r = torch.corrcoef(torch.stack((gene_interactions, gene_expressions)))[0, 1]
                if len(gene_interactions)<=20 or ((gene_interactions == gene_interactions[0]).all() or (gene_expressions == gene_expressions[0]).all()):
                    corr_coeffs.append(0)
                    continue

                count_from=(cell_type_counts.loc[cell_type_counts["cell_type"]==from_type,"counts"].values)[0]      
                count_to=(cell_type_counts.loc[cell_type_counts["cell_type"]==to_type,"counts"].values)[0]
                avg_cnti=avg_cnt[i]
                strength = calcualte_z_neighbor(gene_interactions,expectedi,avg_cnti)
                
                corr_coeffs.append(float(strength))
            results_matrix.append(corr_coeffs)
    
    # Convert results to a tensor of shape (t^2, C)
    results_tensor = np.array(results_matrix)
    results_matrix=np.nan_to_num(results_matrix)
    return results_tensor

z_dir="./z_strength_spatial_neighbor_adapt/"
if not os.path.exists(z_dir):
    os.system("mkdir "+z_dir)
    
results=[]
cnt=0
for samplei in samples:
    print(cnt+1,len(samples))
    cnt=cnt+1
    tmp=calculate_strength_spatial_neighbor_adapt(samplei)
    print(tmp.shape,tmp)
    df=pd.DataFrame(data=tmp,columns=genes,index=cell_type_pair_sequence)
    df.to_csv(z_dir+samplei+".csv")
    results.append(tmp)
    print("spatial neighbor strength:",samplei)

np.save(z_dir+"z_values.npy",np.stack(results,axis=0))

1 8
(484, 960) [[ 1.14207324e-02  0.00000000e+00  5.89413056e-03 ...  1.74048916e-02
   9.49340034e-03  1.03476411e-02]
 [ 7.97073182e-04 -2.79298634e-03  0.00000000e+00 ... -3.04551632e-03
  -8.56017228e-03 -7.81570841e-03]
 [ 1.44081205e-04  1.60970876e-06 -1.67417456e-04 ...  2.89504314e-05
  -1.39706070e-04 -5.10420650e-05]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-7.31564453e-03  1.49438751e-03 -3.34779881e-02 ... -3.85417454e-02
  -4.13757339e-02 -3.18926200e-02]
 [-9.88456793e-03 -2.72431560e-02 -3.66528407e-02 ... -3.55425179e-02
  -1.82338730e-02 -3.17515768e-02]]
spatial neighbor strength: Lung13
2 8
(484, 960) [[ 1.44251408e-02 -9.31885932e-03 -2.21060272e-02 ... -1.19677307e-02
  -5.85750560e-04 -1.06983641e-02]
 [-3.70521285e-03 -8.14495434e-04 -2.40241483e-04 ... -3.86849308e-04
   1.35120237e-04 -3.26794595e-03]
 [ 2.19935505e-03 -4.89860075e-04 -3.85336112e-03 ... -1.10312314e-04
   7.67536229e-04 -4